# Situational Analysis: Teachers' Efficacy in Using Digital Resources for STEM Subjects in Kenyan Schools

## Date: 16/05/2025

## Analyst : Simiyu David Masai
---

# Phase 1: BUSINESS UNDERSTANDING

## 1.1 Project Overview

- This project analyzes the efficacy of teachers in Kenyan Junior and Secondary schools in utilizing digital resources for STEM (Science, Technology, Engineering, and Mathematics) education.
- The analysis is based on data collected through Head of Institution interviews, teacher questionnaires, learner Focus Group Discussions (FGDs), and lesson observations.
- The primary goal is to **understand the current landscape, identify challenges, and provide actionable insights for stakeholders like CEMASTEA (Centre for Mathematics, Science and Technology Education in Africa) and the Ministry of Education, Kenya**.

This research aims to address the following:
* **General Objective:**
- To assess the teachers’ efficacy on the use of digital resources for learning STEM subjects in Kenyan schools.
* **Specific Objectives:**
    1.  To investigate the relationship between teacher training (TPD) in STEM and their ability to effectively use digital resources.
    2.  To establish the influence of the use of digital resources on the teacher's instructional practices in STEM subjects.
    3.  To establish teachers’ awareness on the available digital resources for STEM subjects.
    4.  To assess the extent of support provided to teachers for adopting digital resources in teaching STEM subjects.

**Keywords:** STEM Education, Digital Resources, Teacher Efficacy, Kenya, Educational Technology, Teacher Training, Instructional Practices, CEMASTEA, CBC Kenya.

---

## 1.2 KEY STAKEHOLDERS

1. MINISTRY OF EDUCATION
2. CENTRE OF MATHEMATICS SCIENCE AND TECHNOLOGY EDUCATION IN AFRICA
3. SCHOOL ADMINSTRATORS,TEACHERS AND LEARNERS
4. RESEARCH COMMUNITIES
